In [102]:
# TensorFlow Model !
import os
import shutil
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
from cell import ConvLSTMCell
import sys
module_path = os.path.join("/home/pratik/work/dl/deepvideos/model/../")
if module_path not in sys.path:
    sys.path.append(module_path)
from datasets.batch_generator import datasets
slim = tf.contrib.slim
from tensorflow.python.ops import init_ops
from tensorflow.contrib.layers.python.layers import regularizers
trunc_normal = lambda stddev: init_ops.truncated_normal_initializer(0.0, stddev)
l2_val = 0.00005

In [46]:
# For looped RNN
batch_size = 4
timesteps = 4
conv_data_timesteps = timesteps * 2
shape = [64, 64]  # Image shape
H, W, C = 64, 64, 3
kernel = [5, 5]
channels = 3
filters = [128, 128]  # 2 stacked conv lstm filters

In [47]:
inp = tf.placeholder(tf.float32,(batch_size, conv_data_timesteps, H, W, C))
inp_to_conv_layer = tf.reshape(inp,[-1,H,W,C])

In [48]:
def conv_layer(inp,reuse):
    with tf.variable_scope('conv_before_lstm',reuse=reuse):
        net = slim.conv2d(inp, 128, [7,7], scope='conv_1',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d(net, 256, [5,5], scope='conv_2',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d(net, 512, [5,5], scope='conv_3',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d(net, 256, [5,5], scope='conv_4',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d(net, 128, [7,7], scope='conv_5',weights_initializer=trunc_normal(0.01))
        print net
    return net

In [49]:
def deconv_layer(deconv_input,reuse=None):
    with tf.variable_scope('deconv_after_lstm',reuse=reuse):
        net = slim.conv2d_transpose(deconv_input, 128, [7, 7], scope='deconv_5',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d_transpose(net, 256, [5, 5], scope='deconv_4', weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d_transpose(net, 512, [5, 5], scope='deconv_3',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d_transpose(net, 256, [5, 5], scope='deconv_2',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d_transpose(net, 128, [7, 7], scope='deconv_1',weights_initializer=trunc_normal(0.01))
        print net
        net = slim.conv2d_transpose(net, 3, [7, 7], activation_fn=tf.tanh, scope='deconv_0',weights_initializer=trunc_normal(0.01))
        print net
        return net

In [50]:
deinp = tf.placeholder(tf.float32,(batch_size, 8, H, W, 128))
inp_to_deconv_layer = tf.reshape(deinp,[-1,64,64,128])

In [51]:
inp_to_deconv_layer

<tf.Tensor 'Reshape_1:0' shape=(32, 64, 64, 128) dtype=float32>

In [52]:
deconv_layer(inp_to_deconv_layer)

Tensor("deconv_after_lstm/deconv_5/Relu:0", shape=(32, 64, 64, 128), dtype=float32)
Tensor("deconv_after_lstm/deconv_4/Relu:0", shape=(32, 64, 64, 256), dtype=float32)
Tensor("deconv_after_lstm/deconv_3/Relu:0", shape=(32, 64, 64, 512), dtype=float32)
Tensor("deconv_after_lstm/deconv_2/Relu:0", shape=(32, 64, 64, 256), dtype=float32)
Tensor("deconv_after_lstm/deconv_1/Relu:0", shape=(32, 64, 64, 128), dtype=float32)
Tensor("deconv_after_lstm/deconv_0/Tanh:0", shape=(32, 64, 64, 3), dtype=float32)


<tf.Tensor 'deconv_after_lstm/deconv_0/Tanh:0' shape=(32, 64, 64, 3) dtype=float32>

In [53]:
output_of_conv_layer = conv_layer(inp_to_conv_layer,None)
cB, cH, cW, cC = output_of_conv_layer.get_shape().as_list()
print (cB,cH,cW,cC)
inp_time_based = tf.reshape(output_of_conv_layer, [-1,conv_data_timesteps, cH, cW, cC])
print (inp_time_based)
encoder_input = tf.slice(inp_time_based,[0,0,0,0,0],[batch_size,timesteps,cH,cW,cC])
print (encoder_input)
decoder_input = tf.slice(inp_time_based,[0,timesteps,0,0,0],[batch_size,timesteps,cH,cW,cC])
print (decoder_input)

Tensor("conv_before_lstm/conv_1/Relu:0", shape=(32, 64, 64, 128), dtype=float32)
Tensor("conv_before_lstm/conv_2/Relu:0", shape=(32, 64, 64, 256), dtype=float32)
Tensor("conv_before_lstm/conv_3/Relu:0", shape=(32, 64, 64, 512), dtype=float32)
Tensor("conv_before_lstm/conv_4/Relu:0", shape=(32, 64, 64, 256), dtype=float32)
Tensor("conv_before_lstm/conv_5/Relu:0", shape=(32, 64, 64, 128), dtype=float32)
Tensor("conv_before_lstm/conv_5/Relu:0", shape=(32, 64, 64, 128), dtype=float32)
(32, 64, 64, 128)
Tensor("Reshape_2:0", shape=(4, 8, 64, 64, 128), dtype=float32)
Tensor("Slice:0", shape=(4, 4, 64, 64, 128), dtype=float32)
Tensor("Slice_1:0", shape=(4, 4, 64, 64, 128), dtype=float32)


In [28]:
kernels = [[3,3],[5, 5]]
H, W = 64, 64
with tf.variable_scope('enc_conv_lstm_model'):
    cells = []
    for i, (each_filter,each_kernel) in enumerate(zip(filters,kernels)):
        cell = ConvLSTMCell([H,W], each_filter, each_kernel,reuse=tf.get_variable_scope().reuse)
        cells.append(cell)

    cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)  

In [29]:
cell

In [30]:
zero_state = cell.zero_state(batch_size,dtype=tf.float32)
zero_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/ConvLSTMCellZeroState/zeros:0' shape=(4, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/ConvLSTMCellZeroState/zeros_1:0' shape=(4, 64, 64, 128) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/ConvLSTMCellZeroState_1/zeros:0' shape=(4, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/ConvLSTMCellZeroState_1/zeros_1:0' shape=(4, 64, 64, 128) dtype=float32>))

In [31]:
encoder_output, encoder_state = tf.nn.dynamic_rnn(cell,inputs=encoder_input,initial_state=zero_state)

In [32]:
encoder_input

<tf.Tensor 'Slice:0' shape=(4, 4, 64, 64, 128) dtype=float32>

In [33]:
encoder_output

<tf.Tensor 'rnn/transpose:0' shape=(4, 4, 64, 64, 128) dtype=float32>

In [34]:
encoder_state

(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(4, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(4, 64, 64, 128) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(4, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(4, 64, 64, 128) dtype=float32>))

In [55]:
encoder_output.get_shape().as_list()

[4, 4, 64, 64, 128]

In [ ]:
kernels = [[3,3],[5, 5]]
H, W = 64, 64
with tf.variable_scope('dec_conv_lstm_model'):
    cells = []
    for i, (each_filter,each_kernel) in enumerate(zip(filters,kernels)):
        cell = ConvLSTMCell([H,W], each_filter, each_kernel,reuse=tf.get_variable_scope().reuse)
        cells.append(cell)

    cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)  

## Model 

In [103]:
batch_size = 16
number_of_images_to_show = 4
assert number_of_images_to_show <= batch_size
shape = [64, 64]  # Image shape
H, W = shape
kernels = [[3, 3],[5, 5]]
channels = C = 3
filters = [128, 128]  # 2 stacked conv lstm filters
enc_timesteps = 8 - 1
dec_timesteps = 8
timesteps = enc_timesteps + dec_timesteps
images_summary_timesteps = [0, 2, 5, 7]

# Create a placeholder for videos.
inputs = tf.placeholder(tf.float32, [batch_size, timesteps] + shape + [channels],
                             name="seq2seq_inputs")  # (batch_size, timestep, H, W, C)
outputs_exp = tf.placeholder(tf.float32, [batch_size, dec_timesteps] + shape + [channels],
                                  name="seq2seq_outputs_exp")  # (batch_size, timestep, H, W, C)
teacher_force_sampling = tf.placeholder(tf.float32, [dec_timesteps], name="teacher_force_sampling")
prob_select_teacher = tf.placeholder(tf.float32, shape=(), name="prob_select_teacher")

# model output
model_output = None

# loss
l2_loss = None

# optimizer
optimizer = None

reuse_conv = None
reuse_deconv = None

In [112]:
def conv_layer(conv_input):
        # conv before lstm
        with tf.variable_scope('conv_before_lstm',reuse=True):
            net = slim.conv2d(conv_input, 128, [7,7], scope='conv_1',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d(net, 256, [5,5], scope='conv_2',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d(net, 512, [5,5], scope='conv_3',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d(net, 256, [5,5], scope='conv_4',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d(net, 128, [7,7], scope='conv_5',weights_initializer=trunc_normal(0.01))
            reuse_conv = True
            return net

In [120]:
def deconv_layer(deconv_input):
        with tf.variable_scope('deconv_after_lstm',reuse=True):
            net = slim.conv2d_transpose(deconv_input, 128, [7, 7], scope='deconv_5',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d_transpose(net, 256, [5, 5], scope='deconv_4', weights_initializer=trunc_normal(0.01))
            net = slim.conv2d_transpose(net, 512, [5, 5], scope='deconv_3',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d_transpose(net, 256, [5, 5], scope='deconv_2',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d_transpose(net, 128, [7, 7], scope='deconv_1',weights_initializer=trunc_normal(0.01))
            net = slim.conv2d_transpose(net, 3, [7, 7], activation_fn=tf.tanh, scope='deconv_0',weights_initializer=trunc_normal(0.01))
            reuse_deconv = True
            return net

In [109]:
def enc_lstm_layer(H,W):
        with tf.variable_scope('enc_lstm_model'):
            cells = []
            for i, (each_filter, each_kernel) in enumerate(zip(filters,kernels)):
                cell = ConvLSTMCell([H, W], each_filter, each_kernel,reuse=tf.get_variable_scope().reuse)
                cells.append(cell)

            cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)
            return cell

def dec_lstm_layer(H,W):
        with tf.variable_scope('dec_lstm_model'):
            cells = []
            for i, (each_filter, each_kernel) in enumerate(zip(filters,kernels)):
                cell = ConvLSTMCell([H, W], each_filter, each_kernel,reuse=tf.get_variable_scope().reuse)
                cells.append(cell)

            cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)
            return cell

In [110]:
H, W, C = shape[0], shape[1], channels
input_conv_layer = tf.reshape(inputs, [-1,H,W,C])
output_conv_layer = conv_layer(input_conv_layer)
_, H, W, C = output_conv_layer.get_shape().as_list()
lstm_shaped_input = tf.reshape(output_conv_layer, [-1,timesteps,H,W,C])

In [113]:
input_conv_layer

<tf.Tensor 'Reshape:0' shape=(240, 64, 64, 3) dtype=float32>

In [114]:
output_conv_layer

<tf.Tensor 'conv_before_lstm/conv_5/Relu:0' shape=(240, 64, 64, 128) dtype=float32>

In [115]:
lstm_shaped_input

<tf.Tensor 'Reshape_1:0' shape=(16, 15, 64, 64, 128) dtype=float32>

In [116]:
# slice first part to feed to encoder and second to decoder
encoder_inp = tf.slice(lstm_shaped_input,[0,0,0,0,0],[batch_size,enc_timesteps,H,W,C])
decoder_inp = tf.slice(lstm_shaped_input,[0,enc_timesteps,0,0,0],[batch_size,dec_timesteps,H,W,C])

# dynamic rnn as encoder
encoder_cell = enc_lstm_layer(H,W)
zero_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)
encoder_output, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell,inputs=encoder_inp,initial_state=zero_state)

In [117]:
encoder_final_state

(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(16, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(16, 64, 64, 128) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(16, 64, 64, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(16, 64, 64, 128) dtype=float32>))

In [118]:
encoder_output

<tf.Tensor 'rnn/transpose:0' shape=(16, 7, 64, 64, 128) dtype=float32>

In [119]:
# decoder cell 
decoder_cell = dec_lstm_layer(H,W)
state = encoder_final_state
input_for_first_time = tf.slice(decoder_inp, [0,0,0,0,0], [batch_size,1,H,W,C])
input_for_first_time = tf.squeeze(input_for_first_time,[1])
input_deconv, state = decoder_cell(input_for_first_time,state)
predications = []
deconv_output = deconv_layer(input_deconv)
predications.append(deconv_output)

In [121]:
input_for_first_time

<tf.Tensor 'Squeeze:0' shape=(16, 64, 64, 128) dtype=float32>

In [122]:
input_deconv

<tf.Tensor 'multi_rnn_cell/cell_1/cell_1/conv_lstm_cell/mul_5:0' shape=(16, 64, 64, 128) dtype=float32>

In [123]:
deconv_output

<tf.Tensor 'deconv_after_lstm/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>

In [125]:
for i in range(1,dec_timesteps):
    select_sampling = tf.greater_equal(prob_select_teacher, tf.gather(teacher_force_sampling,i))
    # Conv on actual t_timestep input
    ith_frame = tf.slice(decoder_inp,[0,i,0,0,0],[batch_size,1,64,64,3])
    ith_frame = tf.squeeze(ith_frame,[1])
    conv_output = conv_layer(ith_frame)
    branch_1 = decoder_cell(conv_output, state)
    # Conv on predicated t-1_timestep input
    conv_output = conv_layer(deconv_output)
    branch_2 = decoder_cell(conv_output, state)

    deconv_input, state = tf.cond(select_sampling, lambda: branch_1, lambda: branch_2)
    deconv_output = deconv_layer(deconv_input)
    predications.append(deconv_output)  

In [126]:
predications

[<tf.Tensor 'deconv_after_lstm/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_1/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_2/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_3/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_4/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_5/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_6/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'deconv_after_lstm_7/deconv_0/Tanh:0' shape=(16, 64, 64, 3) dtype=float32>]

In [128]:
model_output = tf.transpose(tf.stack(predications),perm=[1,0,2,3,4])

In [129]:
model_output

<tf.Tensor 'transpose_1:0' shape=(16, 8, 64, 64, 3) dtype=float32>

In [130]:
outputs_exp

<tf.Tensor 'seq2seq_outputs_exp:0' shape=(16, 8, 64, 64, 3) dtype=float32>

In [134]:
def l2_loss(generated_frames, expected_frames):
    losses = []
    for each_scale_gen_frames, each_scale_exp_frames in zip(generated_frames, expected_frames):
        losses.append(tf.nn.l2_loss(tf.subtract(each_scale_gen_frames, each_scale_exp_frames)))
    
    loss = tf.reduce_mean(tf.stack(losses))
    return loss

def gdl_loss(generated_frames, expected_frames, alpha=2):
    """
    difference with side pixel and below pixel
    """
    scale_losses = []
    for i in xrange(len(generated_frames)):
        # create filters [-1, 1] and [[1],[-1]] for diffing to the left and down respectively.
        pos = tf.constant(np.identity(3), dtype=tf.float32)
        neg = -1 * pos
        filter_x = tf.expand_dims(tf.stack([neg, pos]), 0)  # [-1, 1]
        filter_y = tf.stack([tf.expand_dims(pos, 0), tf.expand_dims(neg, 0)])  # [[1],[-1]]
        strides = [1, 1, 1, 1]  # stride of (1, 1)
        padding = 'SAME'

        gen_dx = tf.abs(tf.nn.conv2d(generated_frames[i], filter_x, strides, padding=padding))
        gen_dy = tf.abs(tf.nn.conv2d(generated_frames[i], filter_y, strides, padding=padding))
        gt_dx = tf.abs(tf.nn.conv2d(expected_frames[i], filter_x, strides, padding=padding))
        gt_dy = tf.abs(tf.nn.conv2d(expected_frames[i], filter_y, strides, padding=padding))

        grad_diff_x = tf.abs(gt_dx - gen_dx)
        grad_diff_y = tf.abs(gt_dy - gen_dy)

        scale_losses.append(tf.reduce_sum((grad_diff_x ** alpha + grad_diff_y ** alpha)))

    # condense into one tensor and avg
    return tf.reduce_mean(tf.stack(scale_losses))

def total_loss(generated_frames, expected_frames, lambda_gdl=1.0, lambda_l2=1.0):
      B, T, H, W, C = generated_frames.get_shape().as_list()
      B1, T1, H1, W1, C1 = expected_frames.get_shape().as_list()
      assert (B, T, H, W, C)==(B1, T1, H1, W1, C1),"shape should be equal of gen and exp frames !"
      each_step_gen_frames = []
      each_step_exp_frames = []
      for each_i in range(T):
            input_for_gen = tf.slice(generated_frames, [0,each_i,0,0,0], [B,1,H,W,C])
            input_for_gen = tf.squeeze(input_for_gen,[1])
            each_step_gen_frames.append(input_for_gen)
            
            input_for_exp = tf.slice(expected_frames, [0,each_i,0,0,0], [B,1,H,W,C])
            input_for_exp = tf.squeeze(input_for_exp,[1])
            each_step_exp_frames.append(input_for_exp)

      total_loss_cal = (lambda_gdl * gdl_loss(each_step_gen_frames, each_step_exp_frames) + 
                     lambda_l2 * l2_loss(each_step_gen_frames, each_step_exp_frames))
      return total_loss_cal

In [135]:
l = total_loss(model_output,outputs_exp)

Tensor("Squeeze_25:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_26:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_27:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_28:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_29:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_30:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_31:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_32:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_33:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_34:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_35:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_36:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_37:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_38:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_39:0", shape=(16, 64, 64, 3), dtype=float32)
Tensor("Squeeze_40:0", shape=(16, 64, 64, 3), dtype=float32)


In [136]:
encoder_inp

<tf.Tensor 'Slice:0' shape=(16, 7, 64, 64, 128) dtype=float32>

In [137]:
decoder_inp

<tf.Tensor 'Slice_1:0' shape=(16, 8, 64, 64, 128) dtype=float32>